In [1]:
# pandas
import pandas as pd
from pandas import Series,DataFrame

# numpy, matplotlib, seaborn
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

/home/yuzhong/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/home/yuzhong/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
patient_transcript_df = pd.read_csv("patient_diagnosis.csv", sep=',')
patient_df = pd.read_csv("../raw/trainingSet/training_SyncPatient.csv", sep=',')
labRes_df = pd.read_csv("../raw/trainingSet/training_SyncLabResult.csv", sep=',')
labPan_df = pd.read_csv("../raw/trainingSet/training_SyncLabPanel.csv", sep=',')
labOb_df = pd.read_csv("../raw/trainingSet/training_SyncLabObservation.csv", sep=',')

In [3]:
s1 = labRes_df.groupby('PatientGuid')['LabResultGuid'].apply(list)
s2 = labRes_df.groupby('PatientGuid')['PatientGuid'].apply(list)
s2 = s2.apply(lambda x:x[0])

result_df = pd.concat([s1, s2], axis=1)
result_df['PanelName2011'] = pd.Series(None, index=result_df.index)
result_df['PanelName2012'] = pd.Series(None, index=result_df.index)
result_df['PanelId2011'] = pd.Series(None, index=result_df.index)
result_df['PanelId2012'] = pd.Series(None, index=result_df.index)

patient_lab_df = result_df.merge(patient_df, on='PatientGuid')

for index, value in patient_lab_df.iterrows():
    panel_df = labPan_df[labPan_df['LabResultGuid'].isin(value['LabResultGuid'])]
    
    lst_name_2011 = panel_df[panel_df['ObservationYear']==2011]["PanelName"].tolist()
    lst_name_2012 = panel_df[panel_df['ObservationYear']==2012]["PanelName"].tolist()
        
    if lst_name_2011: patient_lab_df['PanelName2011'][index] = lst_name_2011
    if lst_name_2012: patient_lab_df['PanelName2012'][index] = lst_name_2012

    lst_id_2011 = panel_df[panel_df['ObservationYear']==2011]["LabPanelGuid"].tolist()
    lst_id_2012 = panel_df[panel_df['ObservationYear']==2012]["LabPanelGuid"].tolist()
    
    if lst_id_2011: patient_lab_df['PanelId2011'][index] = lst_id_2011
    if lst_id_2012: patient_lab_df['PanelId2012'][index] = lst_id_2012

/home/yuzhong/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/yuzhong/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/home/yuzhong/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/yuzhong/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:25

In [45]:
patient_lab_df['Observation2011'] = pd.Series(None, index=patient_lab_df.index, dtype=object)
patient_lab_df['Observation2012'] = pd.Series(None, index=patient_lab_df.index, dtype=object)

for index, value in patient_lab_df.iterrows():
    temp_2011 = None
    temp_2012 = None
    
    if type(pd.notnull(value['PanelId2011']))!=bool:
        temp_2011 = [labOb_df[labOb_df['LabPanelGuid']==panelId]['IsAbnormalValue'].sum() for panelId in value['PanelId2011']]
    if type(pd.notnull(value['PanelId2012']))!=bool:
        temp_2012 = [labOb_df[labOb_df['LabPanelGuid']==panelId]['IsAbnormalValue'].sum() for panelId in value['PanelId2012']]
    
    if temp_2011: 
        patient_lab_df['Observation2011'][index] = temp_2011
    if temp_2012: 
        patient_lab_df['Observation2012'][index] = temp_2012

/home/yuzhong/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/yuzhong/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [51]:
patient_lab_df.to_csv('patient_lab.csv', index=False)